# Description

<font size='+2' color='#005F6A'>**Selectable**</font><br>

* Provides a mechanism for selecting items with either **radio**, **multi-select** or **radiox** (unselecteble radio) behavior.
* Uses **Buttons** to represent selectable items.
* Allows for **custom behavior** on first selection and subsequent selections.

# Class

In [ ]:
#| default_exp Selectable

In [ ]:
#| hide
from ipywidgets import Button, HTML, HBox, VBox, Label, Output
from IPython.display import display

In [ ]:
#| export
class Selectable:
    """A class representing a selectable item with customizable behavior."""
    
    def __init__(self, items, behave, selector=None):
        """Initializes the class with given parameters."""
        self.isSelected, self.items, self.behave = False, items, behave
        self.bu_selector = Button(style={'button_color': '#99bfc3'}, layout={'width': '22px', 'height': '22px'})
        self.widget = HBox(children=[self.bu_selector], layout={'min_height': '24px', 'overflow': 'hidden'})
        self.items.append(self)
        self.bu_selector.on_click(self.select)
        self.setInitState()
        self.selector = selector

    def doBehavior(self):
        """Executes the selection behavior based on the specified type."""
        self.posList, lastSelect = [], self.lastSelect()

        if self.behave == 'radiox':
            state = self.items[lastSelect].isSelected
            for item in self.items: 
                item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
            if state:
                self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#99bfc3', False
            else:
                self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#005F6A', True
            self.posList = [p for p in range(len(self.items)) if self.items[p].isSelected]

        elif self.behave == 'radio':
            for item in self.items: 
                item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
            self.bu_selector.style.button_color, self.isSelected, self.posList = '#005F6A', True, [lastSelect]

        elif self.behave == 'multi':
            state = self.items[lastSelect].isSelected
            if state:
                self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#99bfc3', False
            else:
                self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#005F6A', True
            self.posList = [p for p in range(len(self.items)) if self.items[p].isSelected]

        else: 
            raise Exception('unknown behavior: '+self.behave)

    def setItemWidget(self, widget):
        """Sets the widget to display for the item."""
        self.itemWidget, self.widget.children = widget, (*self.widget.children, widget)

    def setInitState(self): 
        raise NotImplementedError
        
    def onSelection(self, posList): 
        raise NotImplementedError

    def select(self, b=None):
        """Handles the click event for the selector button."""
        lastSelect = -1
        # search selected item
        for buttonPos in range(len(self.items)):
            if self.items[buttonPos].bu_selector == self.bu_selector: 
                break
        # when found
        if buttonPos < len(self.items):
            # set isLastSelect in all selectables
            for buttonPos in range(len(self.items)): 
                self.items[buttonPos].isLastSelect = False
            self.isLastSelect = True
        # do selectable specific behavior
        self.doBehavior()
        # call selector
        if self.selector:
            self.selector(self.posList, self.lastSelect())

    def lastSelect(self):
        for buttonPos in range(len(self.items)):
            if self.items[buttonPos].bu_selector == self.bu_selector: 
                break
        if buttonPos < len(self.items): 
            return buttonPos
        else: 
            return -1

In [ ]:
#| hide
output = Output()

class MySelectable(Selectable):
    def setInitState(self): 
        self.setItemWidget(Label(value="initial state"))
    
    def onItemSelect(self, posList, lastSelect=None): 
        output.clear_output()
        with output: 
            print('Your selection:', posList, '- lastSelection', lastSelect, end='')
    
    def __init__(self, items, behave='radio'):
        super().__init__(items, behave, self.onItemSelect)

# example: two sets of Selectable objects
a, b, c = [], [], []

display(HTML(value='<font size=+1>behave: radio'))
display(HBox(children=[MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget,
                       MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget]))

display(HTML(value='<hr><font size=+1>behave: multi'))
display(HBox(children=[MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                       MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                       MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget]))

display(HTML(value='<hr><font size=+1>behave: radiox'))
display(HBox(children=[MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget,
                       MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget,
                       MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget]))

display(output)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()